### Refrence :https://medium.com/analytics-vidhya/fine-tuning-bert-for-amazon-food-reviews-32e474de0e51

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 6.5 MB 48.5 MB/s 
     |████████████████████████████████| 596 kB 4.7 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import glob
import numpy as np
import os
import pandas as pd
import sys
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertModel, BertForSequenceClassification

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [7]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/restaurant-train.csv", delimiter = '\t', header = None )
data.columns = ['Rating', 'text']
data.reset_index(drop=True, inplace=True)
data.head()
data.dropna(axis = 0, inplace = True)

In [8]:
data

,Rating,text
0,4,Thank you thank you thank you !! I want to th...
1,5,A Humane Society store at the Biltmore? Inter...
2,1,Don't buy Nike sneakers if you want to return ...
3,3,I have to say I love most things about Sprouts...
4,5,The tire pressure light came on a day or so ag...
...,...,...
82060,5,"""Smeeks is enough to make any grown adult thro..."
82061,5,"""I don't know how it came to be, but an auto s..."
82062,4,I tried this place for the first time last Fri...
82063,2,Sorry. I'm from Southern California so I'm us...


In [9]:
data = data.sample(frac =0.03)
data['text'] = data['text'].astype(str)

In [10]:
data.head()

,Rating,text
35094,4,Went with family from out of town (there hotel...
7752,4,This place is close to my house and I usually ...
64358,1,"""Here is what I wrote to their corporate offic..."
27494,1,"""My belief has always been that at a restauran..."
22578,4,I knew someone who recommended Arizona Hair Co...


## Check CUDA is installed? Installation of CUDA libraries

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(data
    ['Rating'].unique()), # number of unique labels for our multi-class classification problem
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# refrence taken from https://huggingface.co/docs/transformers/model_doc/auto

In [12]:
class RestaurantReviewsDataset(Dataset):
    
    def __init__(self, df, max_length=512):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length 
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # input=review, label=stars
        review = self.df.loc[idx, 'text']
        # labels are 0-indexed
        label = int(self.df.loc[idx, 'Rating']) - 1
        
        encoded = self.tokenizer(
            review,                      # review to encode
            add_special_tokens=True,
            max_length=self.max_length,  # Truncate all segments to max_length
            padding='max_length',        # pad all reviews with the [PAD] token to the max_length
            return_attention_mask=True,  # Construct attention masks.
            truncation=True
        )
        
        input_ids = encoded['input_ids']
        attn_mask = encoded['attention_mask']
        
        return {
            'input_ids': torch.tensor(input_ids),
            'attn_mask': torch.tensor(attn_mask), 
            'label': torch.tensor(label)
        }

In [13]:
train_dataset, test_dataset = train_test_split(data, test_size=0.2, random_state=1)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=1)

train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

train_set = RestaurantReviewsDataset(train_dataset, 256)
val_set = RestaurantReviewsDataset(val_dataset, 256)
test_set = RestaurantReviewsDataset(test_dataset, 256)

print("# of samples in train set: {}".format(len(train_set)))
print("# of samples in val set: {}".format(len(val_set)))
print("# of samples in test set: {}".format(len(test_set)))

# of samples in train set: 1575
# of samples in val set: 394
# of samples in test set: 493


In [14]:
train_params = {
                'batch_size': 5,
                'shuffle': True,
                'num_workers': 2
                }
val_params = train_params

test_params = {
                'batch_size': 5,
                'shuffle': False,
                'num_workers': 2
              }

train_loader = DataLoader(train_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **test_params)

In [15]:
train_loader

In [16]:
# For weighted Cross Entropy Loss
# Penalize errors higher if they come from a class with lower frequency
star_groups = data.groupby('Rating')
star_distribution = []
for i in range(len(data['Rating'].unique())):
    star_distribution.append(len(star_groups.groups[i+1])/len(data))

star_distribution = torch.tensor(star_distribution, dtype=torch.float32)

weights = 1.0 / star_distribution
weights = weights / weights.sum()


print('{:<20}: {}'.format('Star distribution', star_distribution.tolist()))
print('{:<20}: {}'.format('Weights', weights.tolist()))

Star distribution   : [0.08042242377996445, 0.09504467993974686, 0.1730300635099411, 0.3586514890193939, 0.2928513288497925]
Weights             : [0.3558976352214813, 0.30114415287971497, 0.16541720926761627, 0.0798049122095108, 0.09773611277341843]


## Define the optimizer

In [17]:
loss_function = torch.nn.CrossEntropyLoss(weight=weights.to(device), reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-05)

In [19]:
def calculate_accuracy(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [20]:
for data in train_loader:
        print(data)
        break

{'input_ids': tensor([[ 101, 2009, 2003,  ...,    0,    0,    0],
        [ 101, 1000, 2023,  ..., 2017, 2356,  102],
        [ 101, 2017, 2323,  ..., 2903, 1045,  102],
        [ 101, 2028, 1997,  ...,    0,    0,    0],
        [ 101, 2096, 6575,  ..., 2993, 2002,  102]]), 'attn_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'label': tensor([3, 0, 3, 4, 2])}


## Traning method of bert model

In [27]:
def train(epoch):
    # number of batches run by model
    nb_tr_steps = 0
    # number of training examples run by model
    nb_tr_examples = 0
    # number of examples classified correctly by model
    n_correct = 0
    tr_loss = 0
    model.train()

    for batch, data in enumerate(train_loader):
        input_ids = data['input_ids'].to(device)
        mask = data['attn_mask'].to(device)
        labels = data['label'].to(device)

        outputs = model(input_ids, mask)
        loss = loss_function(outputs[0], labels)
        tr_loss += loss.item()

        # gets labels with highest probabilities and their corresponding indices
        big_val, big_idx = torch.max(outputs[0].data, dim=1)
        n_correct += calculate_accuracy(big_idx, labels)

        nb_tr_steps += 1
        nb_tr_examples+=labels.size(0)
        
        if batch % 100 == 0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print("Batch {} of epoch {} complete.".format(batch, epoch+1))
            print(f"Training Loss: {loss_step}   Training Accuracy: {accu_step}")
            if not os.path.exists('outputFolder'):
              os.makedirs('outputFolder')


            # Since a single epoch could take well over hours, we regularly save the model even during evaluation of training accuracy.
            torch.save(model.state_dict(), os.path.join('/content/gdrive/MyDrive/', 'outputFolder', 'model_checkpoint.dat'))
            print("Saving model checkpoint at", os.path.join('/content/gdrive/MyDrive/', 'outputFolder', 'model_checkpoint.dat'))
            # Since a single epoch could take well over hours, we regularly save the model even during evaluation of training accuracy.
            #torch.save(net.state_dict(), os.path.join(config["outputFolder"], config["outputFileName"]))
            #print("Iteration {} of epoch {} complete. Loss : {} Accuracy : {}".format(it+1, ep+1, loss.item(), acc))

        optimizer.zero_grad()
        loss.backward()
        # When using GPU
        optimizer.step()

    print('\n*****\n')
    print(f'The Total Accuracy for Epoch {epoch+1}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss: {epoch_loss}")
    print(f"Training Accuracy: {epoch_accu}\n")

    # Evaluate model after training it on this epoch
    #validate(model, val_loader)

    torch.save(model.state_dict(), os.path.join('/content/gdrive/MyDrive/', 'outputFolder', 'model_checkpoint.dat'))
    model.save_pretrained(os.path.join('/content/gdrive/MyDrive/outputFolder/', 'Model_V3', str(epoch+1)))
    print("Saving checkpoint at ", os.path.join('/content/gdrive/MyDrive/', 'outputFolder', 'model_checkpoint.dat'))
    print("Saving model at ", os.path.join('/content/gdrive/MyDrive/outputFolder/', 'Model_V3', str(epoch+1)), '\n\n================================================\n')

    return

## Traning BERT model

In [28]:
for epoch in range(4):
    train(epoch)

Batch 0 of epoch 1 complete.
Training Loss: 0.5048931837081909   Training Accuracy: 80.0
Saving model checkpoint at /content/gdrive/MyDrive/outputFolder/model_checkpoint.dat
Batch 100 of epoch 1 complete.
Training Loss: 0.5230938085530064   Training Accuracy: 78.61386138613861
Saving model checkpoint at /content/gdrive/MyDrive/outputFolder/model_checkpoint.dat
Batch 200 of epoch 1 complete.
Training Loss: 0.4795739596978349   Training Accuracy: 80.0
Saving model checkpoint at /content/gdrive/MyDrive/outputFolder/model_checkpoint.dat
Batch 300 of epoch 1 complete.
Training Loss: 0.49561401441742814   Training Accuracy: 79.46843853820597
Saving model checkpoint at /content/gdrive/MyDrive/outputFolder/model_checkpoint.dat

*****

The Total Accuracy for Epoch 1: 79.55555555555556
Training Loss: 0.4943000292257657
Training Accuracy: 79.55555555555556

Saving checkpoint at  /content/gdrive/MyDrive/outputFolder/model_checkpoint.dat
Saving model at  /content/gdrive/MyDrive/outputFolder/Model_V

##Summary of model

In [23]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=device, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1695 MB |    3092 MB |    7793 GB |    7791 GB |\n|       from large pool |    1693 MB |    3068 MB |    7759 GB |    7757 GB |\n|       from small pool |       1 MB |      25 MB |      33 GB |      33 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1695 MB |    3092 MB |    7793 GB |    7791 GB |\n|       from large pool |    1693 MB |    3068 MB |